In [17]:
import json
from graphviz import Digraph

# Load the JSON file
with open('form.json', 'r') as f:
    data = json.load(f)

fields = data['fields']

# Exclude fields with fieldType 'divider' or 'text_only'
excluded_types = {'divider', 'text_only'}
filtered_fields = [field for field in fields if field.get('fieldType') not in excluded_types]

# Create a mapping from formFieldId to label or fieldType if label is missing
field_labels = {}
branchid_to_fieldid = {}
for field in filtered_fields:
    branch_id = field.get('branchId')
    field_id = field.get('formFieldId')
    label = field.get('label', '').strip()
    if not label:
        label = field.get('fieldType', '')
    if field_id:
        field_labels[field_id] = label
    if branch_id and field_id:
        branchid_to_fieldid.setdefault(branch_id, []).append(field_id)

# Create a Graphviz Digraph
dot = Digraph(comment='Form Flowchart', format='png')
dot.attr(rankdir='LR', size='20,16')
dot.graph_attr['dpi'] = '1600'

# Add nodes for each field
for field_id, label in field_labels.items():
    dot.node(field_id, label)

# Add edges for branching options
for field in filtered_fields:
    field_id = field.get('formFieldId')
    options = field.get('options', [])
    for option in options:
        branch_id = option.get('branchId')
        option_label = option.get('label', '')
        if branch_id and field_id:
            # Add option node
            option_node_id = f"{field_id}_{option_label}"
            dot.node(option_node_id, option_label, shape='box', style='filled', color='lightblue')
            dot.edge(field_id, option_node_id)
            # Connect option to all fields with matching branchId
            for target_field_id in branchid_to_fieldid.get(branch_id, []):
                dot.edge(option_node_id, target_field_id)

# Save and render the diagram
dot.render('form_flowchart', view=True)
print("Flowchart generated as form_flowchart_9th.png")

Flowchart generated as form_flowchart_9th.png
